# Nonlinear Dynamics

Presentation of the neural model supported by kernel regression on nonlinear dynamical dataset.
Dynamics are represented using delay-line, which effectively makes the dataset MISO, with 16 input dimensions (the length of the delay-line).

*Note*: to see how dataset was generated, go to `dataset.ipynb`.

In [ ]:
import sys

sys.path.append("..")  # we run from subdirectory, so to access sources append repo root to path

In [ ]:
import pandas as pd
import numpy as np
import torch
import seaborn as sns
from sklearn import metrics
from matplotlib import pyplot as plt

from pydentification.data.datamodules.simulation import SimulationDataModule
from pydentification.data.process import unbatch
from pydentification.models.nonparametric import kernels
from pydentification.models.nonparametric.memory import ExactMemoryManager
from pydentification.training.bounded.module import BoundedSimulationTrainingModule

In [ ]:
sns.set()

# Dataset

Dataset contains 3 columns, independent time-index, inputs and outputs of the dynamical system.

In [ ]:
dataset_path = "../data/csv/nonlinear-dynamics.csv"
plot_path = "../data/plots/nonlinear-dynamics/"
model_path = "../models/nonlinear-dynamics-network.pt"

train_size = 50_000

In [ ]:
dataset = pd.read_csv(dataset_path)
dataset.head(3)

In [ ]:
_ = plt.figure(figsize=[10, 6])

_ = plt.scatter(dataset["t"].iloc[:train_size], dataset["y"].iloc[:train_size], s=1)
_ = plt.scatter(dataset["t"].iloc[train_size:], dataset["y"].iloc[train_size:], s=1)

_ = plt.legend(["Train Outputs", "Test Outputs"])
_ = plt.savefig(fr"{plot_path}/system-outputs.png")

In [ ]:
_ = plt.figure(figsize=[10, 6])

_ = plt.scatter(dataset["t"].iloc[:train_size], dataset["u"].iloc[:train_size], s=1)
_ = plt.scatter(dataset["t"].iloc[train_size:], dataset["u"].iloc[train_size:], s=1)

_ = plt.legend(["Train Inputs", "Test Inputs"])
_ = plt.savefig(fr"{plot_path}/system-inputs.png")

# Dataset

In [ ]:
dm = SimulationDataModule.from_csv(
    dataset_path=dataset_path,
    input_columns=["u"],
    output_columns=["y"],
    test_size=len(dataset) - train_size,
    batch_size=64,
    validation_size=0.0,
    shift=1,
    forward_input_window_size=16,  # delay-line length
    forward_output_window_size=16,
    forward_output_mask=15,
)

In [ ]:
# setup the data for prediction
dm.setup("fit")
dm.setup("predict")

In [ ]:
for x, y in dm.train_dataloader():
    print(x.shape, y.shape)
    break

In [ ]:
for x, y in dm.test_dataloader():
    print(x.shape, y.shape)
    break

# Model

Create the model from neural network we have trained before.

The settings for kernel regression are selected using hyper-parameter search, which are the best we found for this problem.

In [ ]:
network = torch.load(model_path)

In [ ]:
model = BoundedSimulationTrainingModule(
    network=network,
    optimizer=torch.optim.Adam(network.parameters()),  # not needed
    lr_scheduler=None,
    bound_during_training=False,
    bound_crossing_penalty=0.0,
    bandwidth=0.5,
    kernel=kernels.box_kernel,
    memory_manager=ExactMemoryManager(),  # using exact memory manager is performant enough in 16 dimensions
    lipschitz_constant=1,
    delta=0.1,
    noise_variance="estimate",  # assume we do not know the variance
    k=32,
    p=2,
    memory_device="cpu",
    predict_device="cpu",
)

In [ ]:
x, y = unbatch(dm.train_dataloader())
x.shape, y.shape

In [ ]:
model.prepare(x, y)

In [ ]:
outputs = model.predict_datamodule(dm, with_targets=True)
type(outputs)

In [ ]:
outputs.keys()

In [ ]:
_ = plt.figure(figsize=[10, 6])

# a few time-samples might be lost due to windowing, slice time index from start of test to last available prediction
t = dataset["t"].iloc[train_size : train_size + len(outputs["targets"])].values
index = np.argsort(t)
# create time index sorting predictions, so we can use line plot
t = t[index]

_ = plt.scatter(t, outputs["targets"][index], s=5)
_ = plt.plot(t, outputs["nonparametric_predictions"][index], c="g")
_ = plt.plot(t, outputs["network_predictions"].numpy().flatten()[index], c="r")
_ = plt.fill_between(t, outputs["lower_bound"].numpy().flatten()[index], outputs["upper_bound"].numpy().flatten()[index], color="b", alpha=0.4)

_ = plt.legend(["Training Data", "Kernel Regression Predictions", "Network Predictions"])
_ = plt.savefig(fr"{plot_path}/predictions.png")

In [ ]:
def range_ratio_error(error, y_true):
    return error / (y_true.max() - y_true.min())

def report(outputs, targets):
    rmse_network = metrics.mean_squared_error(y_true=targets, y_pred=outputs["network_predictions"].numpy().flatten(), squared=False)
    rmse_nonparametric = metrics.mean_squared_error(y_true=targets, y_pred=outputs["nonparametric_predictions"].numpy().flatten(), squared=False)
    rmse_bound = metrics.mean_squared_error(y_true=targets, y_pred=outputs["lower_bound"].numpy().flatten(), squared=False)

    print(f"RMSE NET:    {rmse_network:.4f}")
    print(f"RMSE KRE:    {rmse_nonparametric:.4f}")
    print(f"RMSE BOUNDS: {rmse_bound:.4f}", end="\n\n")
    print(f"RRR NET:     {range_ratio_error(error=rmse_network, y_true=targets):.2%}")
    print(f"RRR KRE:     {range_ratio_error(error=rmse_nonparametric, y_true=targets):.2%}")
    print(f"RRR BOUNDS:  {range_ratio_error(error=rmse_bound, y_true=targets):.2%}")

In [ ]:
report(outputs, outputs["targets"].numpy().flatten())